In [0]:
pip install lightgbm

In [0]:
pip install hyperopt

In [0]:
%pip install databricks-feature_engineering

In [0]:
%restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import os
from datetime import datetime, date 
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
import numpy as np
from mlflow.models.signature import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
from mlflow.tracking import MlflowClient
import requests
from sklearn.metrics import accuracy_score

In [0]:
import pandas as pd

In [0]:
df= pd.read_csv("/Volumes/ispl_databricks/default/training/ff_bd/filtered_data_with_type.csv")
df.shape

In [0]:
ref_columns = ref_df.columns.tolist()

In [0]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')

In [0]:
df['status_date'] = pd.to_datetime(df['status_date'], errors='coerce')
df['month_year'] = df['status_date'].dt.strftime('%m-%Y')

In [0]:
def age(born): 
    try:
        born = datetime.strptime(born, "%d-%m-%Y").date() 
        today = date.today() 
        age = today.year - born.year - ((today.month,  
                                        today.day) < (born.month,  
                                                        born.day))
        if age>=18:
            return age
        else:
            return (-1) 
    except:
        return -1
    
df['bue_age'] =df['bue_dob'].apply(age)

In [0]:
df['bue_perc_no_of_open_loans']=round(df['bue_no_of_open_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_open_loans_lst_6months']=round(df['bue_no_of_open_loans_lst_6months']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_cc_loans']=round(df['bue_no_of_cc_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_cc_open_loans']=round(df['bue_no_of_cc_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_auto_open_loans']=round(df['bue_no_of_auto_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_consumer_loans']=round(df['bue_no_of_consumer_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_consumer_open_loans']=round(df['bue_no_of_consumer_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_personal_loans']=round(df['bue_no_of_personal_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_gold_loans']=round(df['bue_no_of_gold_loans']/df['bue_no_of_loans'],2)

In [0]:
df_num = df.select_dtypes(include=['float64','int64','int32'])
df_num.fillna(0,inplace=True)

In [0]:
df_num.shape

In [0]:
df_cat = df.select_dtypes(include=['object','boolean','category'])

In [0]:
base_df2 = pd.concat([df_num,df_cat],axis=1)
base_df2.shape

In [0]:
null_percentages = base_df2.isnull().mean() * 100
columns_to_drop = null_percentages[null_percentages >= 40].index
base_df2.drop(columns=columns_to_drop, inplace=True)
len(columns_to_drop)

In [0]:
base_df2.shape

In [0]:
base_4=base_df2.drop(columns={'bue_dob','bue_name','imsi'})

In [0]:
base_4 = base_4[ref_columns]

In [0]:
base_4.dtypes

In [0]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in base_4.columns:
    dtype = base_4[col].dtype

    if col == "loan_id":
        base_4[col] = base_4[col].astype(object)
        continue

    if dtype.name in ["category", "object"]:
        le = LabelEncoder()
        base_4[col] = le.fit_transform(
            base_4[col].astype(str)
        )
        label_encoders[col] = le

In [0]:
list1 = base_4.dtypes.tolist()

In [0]:
list1

In [0]:
spark_df = spark.createDataFrame(base_4)

In [0]:
spark_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable('ispl_databricks.model_logs.bd_final_inference_data')